# Importing modules

In [156]:
import pandas as pd
import numpy as np

# Importing Data

In [157]:
file1 = pd.read_csv("Data/file1.csv")
file2 = pd.read_csv("Data/file2.csv")
file3 = pd.read_csv("Data/file3.csv")


# Checking the imported data

First let's check if the shape of the 3 files is compatible.

In [158]:
print(f'The shape of file1 is {file1.shape}')
print(f'The shape of file2 is {file2.shape}')
print(f'The shape of file3 is {file3.shape}')

print(f'The number of rows of the df after concatenation should be: {file1.shape[0] + file2.shape[0] + file3.shape[0]}')


The shape of file1 is (4008, 11)
The shape of file2 is (996, 11)
The shape of file3 is (7070, 11)
The number of rows of the df after concatenation should be: 12074


They are. Now let's check if the headings are the same in order to avoid generating extra columns.

In [159]:
print(set(file1.columns) ^ set(file2.columns))
print(set(file1.columns) ^ set(file3.columns))
print(set(file2.columns) ^ set(file3.columns))

set()
{'Gender', 'GENDER', 'State', 'ST'}
{'Gender', 'GENDER', 'State', 'ST'}


Before concatenaiting the data I will change the heading of file3 to be the same as in file1 and 2

In [160]:
file3.rename(columns={"State":"ST", "Gender": "GENDER"}, inplace=True)

ca_df = pd.concat([file1,file2,file3]).reset_index(drop=True)

Let's check everything looks right in terms of importing.

In [161]:
ca_df

,Customer,ST,GENDER,Education,Customer Lifetime Value,Income,Monthly Premium Auto,Number of Open Complaints,Policy Type,Vehicle Class,Total Claim Amount
0,RB50392,Washington,NaN,Master,NaN,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934
1,QZ44356,Arizona,F,Bachelor,697953.59%,0.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935
2,AI49188,Nevada,F,Bachelor,1288743.17%,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247
3,WW63253,California,M,Bachelor,764586.18%,0.0,106.0,1/0/00,Corporate Auto,SUV,529.881344
4,GA49547,Washington,M,High School or Below,536307.65%,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323
...,...,...,...,...,...,...,...,...,...,...,...
12069,LA72316,California,M,Bachelor,23405.98798,71941.0,73.0,0,Personal Auto,Four-Door Car,198.234764
12070,PK87824,California,F,College,3096.511217,21604.0,79.0,0,Corporate Auto,Four-Door Car,379.200000
12071,TD14365,California,M,Bachelor,8163.890428,0.0,85.0,3,Corporate Auto,Four-Door Car,790.784983
12072,UP19263,California,M,College,7524.442436,21941.0,96.0,0,Personal Auto,Four-Door Car,691.200000


# Standardizing Headings

The headings are standarized using a function.

In [162]:
def stdrz_headings(heading):
    """
    Returns a Pandas Index object containing a standarized heading, i.e lower case and " " replaced by "_"
    
    Parameters
    ----------
    columns_list : iterable
        The df heading.

    Returns
    -------
    Pandas Index Object containing the standarized heading

    """  
    return [clabel.lower().replace(" ", "_") for clabel in heading]


ca_df.columns = stdrz_headings(ca_df.columns)   


# Deleting and rearranging columns

We drop the column customer, as we dont need it for our analysis.

In [163]:
ca_df.drop(columns="customer", inplace=True)

Let's check how the data looks up to now.

In [164]:
ca_df.head()

,st,gender,education,customer_lifetime_value,income,monthly_premium_auto,number_of_open_complaints,policy_type,vehicle_class,total_claim_amount
0,Washington,NaN,Master,NaN,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934
1,Arizona,F,Bachelor,697953.59%,0.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935
2,Nevada,F,Bachelor,1288743.17%,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247
3,California,M,Bachelor,764586.18%,0.0,106.0,1/0/00,Corporate Auto,SUV,529.881344
4,Washington,M,High School or Below,536307.65%,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323


# Correcting data types

First let's show the datatypes we have in our dataframe.

In [165]:
ca_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12074 entries, 0 to 12073
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   st                         9137 non-null   object 
 1   gender                     9015 non-null   object 
 2   education                  9137 non-null   object 
 3   customer_lifetime_value    9130 non-null   object 
 4   income                     9137 non-null   float64
 5   monthly_premium_auto       9137 non-null   float64
 6   number_of_open_complaints  9137 non-null   object 
 7   policy_type                9137 non-null   object 
 8   vehicle_class              9137 non-null   object 
 9   total_claim_amount         9137 non-null   float64
dtypes: float64(3), object(7)
memory usage: 943.4+ KB


Customer_lifetime_value and number_of_open_complaints should be numeric. Let's take a deeper look into these two.

In [166]:
print(set([type(item) for item in ca_df.customer_lifetime_value]))
print(set([type(item) for item in ca_df.number_of_open_complaints]))


{<class 'float'>, <class 'str'>}
{<class 'int'>, <class 'float'>, <class 'str'>}


We need to convert into float/integer the strings. In addition:

- Customer_lifetime_value: some of the data ends with symbol %
- number_of_open_complaints: some of the data is presented in the form n1/n2/n3. Analyzing it we can figure out that the number of complaints is actually n2 (n1 seems to be a control number taking only values 0 or 1 and n3 is always 0)

The option of mapping a function in the corresponding data series has been explored below. I will use a lambda function instead of a dedicated function because the purpose of the function is very specific and it will unlikely to be reused.

In [136]:
#If the value is a string
## strip the symbol %, convert into float and round to 0 decimals
## Else check if the valu is Nan
### if yes return value
### if not round to 0 decimals

ca_df.customer_lifetime_value = ca_df.customer_lifetime_value.apply(lambda x: np.round(float(x.strip("%"))*100) if type(x)==str else np.round(x) if x==x else x)

In [137]:
# If the value is a string of the form n/n/n
## Split it into a list, convert it into array of int and apply sum()
## Else return the value

ca_df.number_of_open_complaints = ca_df.number_of_open_complaints.apply(lambda x: np.array(x.split("/"), dtype=int)[1] if type(x)==str else x)

Let us see the result.

In [81]:
ca_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12074 entries, 0 to 12073
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   st                         9137 non-null   object 
 1   gender                     9015 non-null   object 
 2   education                  9137 non-null   object 
 3   customer_lifetime_value    9130 non-null   float64
 4   income                     9137 non-null   float64
 5   monthly_premium_auto       9137 non-null   float64
 6   number_of_open_complaints  9137 non-null   float64
 7   policy_type                9137 non-null   object 
 8   vehicle_class              9137 non-null   object 
 9   total_claim_amount         9137 non-null   float64
dtypes: float64(5), object(5)
memory usage: 943.4+ KB


# Filtering data and Correcting typos

Columns gender and st have obvious problems that need to be cleaned.

In [82]:
print(ca_df.gender.value_counts())
print()
print(ca_df.st.value_counts())

F         4560
M         4368
Male        40
female      30
Femal       17
Name: gender, dtype: int64

California    3032
Oregon        2601
Arizona       1630
Nevada         882
Washington     768
Cali           120
AZ              74
WA              30
Name: st, dtype: int64


- column st: we will reduce the data to the following unique values [Washington, California, Arizona, Oregon, Nevada]
- column gender: we will reduce the data to the following unique values [M, F]

For this we create a clean_series() function and a clean_one_series() helper function. I decided to use generic functions here because I think they could be used in the future for other projects. The version with a dedicated functions is in a commented cell at the end of this section.

In [83]:
def clean_series(df, columns_list, cleaning_dictionary):
    """
    Modifies in place the target columns of a dataframe, reducing their redundant values into unique values following the pairs {redundant_value : unique_value} provided in the cleaning_dictionary.
    
    
    Parameters
    ----------
    df : Pandas Dataframe
        The data frame to be cleaned
    columns_list : List
        A list of strings with the labels of the target columns
    cleaning_dictionary : Dictionary
        A dictionary of the form {redundant_value : unique_value}

    Returns
    -------
    None.

    The function modifies inplace the provided df with the new cleaned values.

    """
    for column in columns_list:

        df[column] = list(map(clean_one_serie, df[column]))



def clean_one_serie(item, cleaning_dictionary = cleaning_dictionary):

    try:

        return cleaning_dictionary[item]

    except:

        return item


In [84]:
cleaning_dictionary = {"female": "F", "Femal":"F", "Male": "M", "Cali": "California", "AZ":"Arizona", "WA": "Washington"}

clean_series(ca_df, ["st", "gender"], cleaning_dictionary)

In [85]:
print(ca_df.gender.value_counts())
print(ca_df.st.value_counts())

F    4607
M    4408
Name: gender, dtype: int64
California    3152
Oregon        2601
Arizona       1704
Nevada         882
Washington     798
Name: st, dtype: int64


In [86]:
# Cleaning of the column state. First we Define function clean_state() and then we map it to the column

# def clean_state(x):

    # return "California" if x=="Cali" else "Arizona" if x=="AZ" else "Washington" if x=="WA" else x

# map it to column ca_df.state
# ca_df.state = list(map(clean_state, ca_df.state))


In [87]:
# Cleaning of column gender, this time with map and a lambda function
# ca_df.gender = list(map(lambda x: "F" if x in ["female", "Femal"] else "M" if x=="Male" else x, ca_df.gender))

# Removing Duplicates

Here we remove duplicate row.
There were (12074 - 8876) = 3198 duplicated rows

In [88]:
ca_df.drop_duplicates(inplace=True)
ca_df

,st,gender,education,customer_lifetime_value,income,monthly_premium_auto,number_of_open_complaints,policy_type,vehicle_class,total_claim_amount
0,Washington,NaN,Master,NaN,0.0,1000.0,0.0,Personal Auto,Four-Door Car,2.704934
1,Arizona,F,Bachelor,697954.0,0.0,94.0,0.0,Personal Auto,Four-Door Car,1131.464935
2,Nevada,F,Bachelor,1288743.0,48767.0,108.0,0.0,Personal Auto,Two-Door Car,566.472247
3,California,M,Bachelor,764586.0,0.0,106.0,0.0,Corporate Auto,SUV,529.881344
4,Washington,M,High School or Below,536308.0,36357.0,68.0,0.0,Personal Auto,Four-Door Car,17.269323
...,...,...,...,...,...,...,...,...,...,...
12069,California,M,Bachelor,23406.0,71941.0,73.0,0.0,Personal Auto,Four-Door Car,198.234764
12070,California,F,College,3097.0,21604.0,79.0,0.0,Corporate Auto,Four-Door Car,379.200000
12071,California,M,Bachelor,8164.0,0.0,85.0,3.0,Corporate Auto,Four-Door Car,790.784983
12072,California,M,College,7524.0,21941.0,96.0,0.0,Personal Auto,Four-Door Car,691.200000


# Replacing null values


## Numeric Columns

For the numeric values we will replace nan values with the mean. The following functions have been created:

- replace_by_mean()
- nan_counter() -only for checking purposes-



In [89]:
# Function to calculate the mean of a column and replace the Nan values with it

def replace_by_mean(df, columns_list):
    """
    Modifies in place the target columns of a dataframe, replacing their Nan values by the column mean.
    
    Parameters
    ----------
    df : Pandas dataframe
        The data frame to be cleaned
    columns_list : List
        A list of strings with the labels of the target columns

    Returns
    -------
    None.

    The function modifies inplace the provided df.

    """

    for column in columns_list:

        mean = np.mean(df[column])
        df[column] = df[column].fillna(mean)

# Function to store in dictionary the number of nan values per column

def nan_counter(df):
    
    """
    Returns a dictionary containing the number of nan values per column (for dataframe df)

    Parameters
    ----------
    df : Pandas dataframe

    Returns
    -------
    remaining_nan : Dictionary
        Contains the number of nan values in each column of the dataframe

    """
    
    remaining_nan = {}

    for column in df.columns:

        remaining_nan[column] = df[column][df[column].isna() == True].size

    return remaining_nan


columns_list = ["customer_lifetime_value", "income", "monthly_premium_auto", "number_of_open_complaints", "total_claim_amount"]

# nan values in numeric columns replaced by the mean in numeric columns
replace_by_mean(ca_df, columns_list)
print(nan_counter(ca_df))

{'st': 1, 'gender': 123, 'education': 1, 'customer_lifetime_value': 0, 'income': 0, 'monthly_premium_auto': 0, 'number_of_open_complaints': 0, 'policy_type': 1, 'vehicle_class': 1, 'total_claim_amount': 0}


Now that we don't have Nan values we can convert floats to integer if appropiate. In this case all the numeric columns can be converted to int without following significant precision. Again we define a function for this.

In [91]:
def float_to_int(df, columns_list):
    """
    Modifies in place the target columns of a dataframe, converting float values into int.
    
    Parameters
    ----------
    df : Pandas Dataframe
        
    columns_list : List
        A list of strings with the labels of the target columns

    Returns
    -------
    None.

    The function modifies inplace the provided df.

    """

    for column in columns_list:

        df[column] = list(map(lambda x: int(round(x)), df[column]))   

In [94]:
float_to_int(ca_df, columns_list)
ca_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8876 entries, 0 to 12073
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   st                         8875 non-null   object
 1   gender                     8753 non-null   object
 2   education                  8875 non-null   object
 3   customer_lifetime_value    8876 non-null   int64 
 4   income                     8876 non-null   int64 
 5   monthly_premium_auto       8876 non-null   int64 
 6   number_of_open_complaints  8876 non-null   int64 
 7   policy_type                8875 non-null   object
 8   vehicle_class              8875 non-null   object
 9   total_claim_amount         8876 non-null   int64 
dtypes: int64(5), object(5)
memory usage: 762.8+ KB


### What about the income column?

The income columns has more than 2,000 values=0. Does that make sense?

In [16]:
ca_df.income[ca_df.income == 0].size

2256

It coluld make sense. It is young people that doesn´t have a job yet. The insurance is paid by the parents.

In [17]:
ca_df.education.value_counts()

Bachelor                2652
College                 2593
High School or Below    2549
Master                   727
Doctor                   330
Bachelors                 24
Name: education, dtype: int64

561 people with education = Doctor have income = 0. The total of people with Doctor education is 3630. The result is not conclusive, still not sure if the zeros are intended or if it is missing data.

For the purpose of this study I will assume that the 0 values are intended.

In [18]:
print(ca_df[(ca_df.income == 0) & (ca_df.education == "Doctor")].size)
print(ca_df[(ca_df.education == "Doctor")].size)

510
3300


## Categorical columns

Nan value rows dropped in those column with only one occurrence. In the gender column Nan values replace by "U" (for Unknown).

In [97]:
print(nan_counter(ca_df))

{'st': 1, 'gender': 123, 'education': 1, 'customer_lifetime_value': 0, 'income': 0, 'monthly_premium_auto': 0, 'number_of_open_complaints': 0, 'policy_type': 1, 'vehicle_class': 1, 'total_claim_amount': 0}


In [99]:
ca_df.gender = list(map(lambda x: "U" if x != x else x, ca_df.gender))
ca_df.dropna(inplace=True)

In [100]:
print(nan_counter(ca_df))

{'st': 0, 'gender': 0, 'education': 0, 'customer_lifetime_value': 0, 'income': 0, 'monthly_premium_auto': 0, 'number_of_open_complaints': 0, 'policy_type': 0, 'vehicle_class': 0, 'total_claim_amount': 0}


Let us see how the data looks now.

In [101]:
ca_df

,st,gender,education,customer_lifetime_value,income,monthly_premium_auto,number_of_open_complaints,policy_type,vehicle_class,total_claim_amount
0,Washington,U,Master,185590,0,1000,0,Personal Auto,Four-Door Car,3
1,Arizona,F,Bachelor,697954,0,94,0,Personal Auto,Four-Door Car,1131
2,Nevada,F,Bachelor,1288743,48767,108,0,Personal Auto,Two-Door Car,566
3,California,M,Bachelor,764586,0,106,0,Corporate Auto,SUV,530
4,Washington,M,High School or Below,536308,36357,68,0,Personal Auto,Four-Door Car,17
...,...,...,...,...,...,...,...,...,...,...
12069,California,M,Bachelor,23406,71941,73,0,Personal Auto,Four-Door Car,198
12070,California,F,College,3097,21604,79,0,Corporate Auto,Four-Door Car,379
12071,California,M,Bachelor,8164,0,85,3,Corporate Auto,Four-Door Car,791
12072,California,M,College,7524,21941,96,0,Personal Auto,Four-Door Car,691


# Bucketing the data

In [19]:
ca_df.insert(1, "region", list(map(lambda x: "west" if x=="California" else "east" if x=="Washington" else "north" if x=="Oregon" else "central", ca_df.state)))

# Standardizing the data

In [20]:
ca_df_standarized = pd.DataFrame(list(map(lambda x: list(map(lambda y: y.lower() if type(y)==str else y, x)), ca_df.values)))
ca_df_standarized.columns = ca_df.columns



In [21]:
ca_df_standarized

,state,region,gender,education,customer_lifetime_value,income,monthly_premium_auto,number_open_complaints,policy_type,vehicle_class,total_claim_amount
0,washington,east,u,master,185590,0,1000,0,personal auto,four-door car,3
1,arizona,central,f,bachelor,697954,0,94,0,personal auto,four-door car,1131
2,nevada,central,f,bachelor,1288743,48767,108,0,personal auto,two-door car,566
3,california,west,m,bachelor,764586,0,106,0,corporate auto,suv,530
4,washington,east,m,high school or below,536308,36357,68,0,personal auto,four-door car,17
...,...,...,...,...,...,...,...,...,...,...,...
8870,california,west,m,bachelor,23406,71941,73,0,personal auto,four-door car,198
8871,california,west,f,college,3097,21604,79,0,corporate auto,four-door car,379
8872,california,west,m,bachelor,8164,0,85,3,corporate auto,four-door car,791
8873,california,west,m,college,7524,21941,96,0,personal auto,four-door car,691


# Getting numerical columns

In [23]:
ca_df_standarized._get_numeric_data()

,customer_lifetime_value,income,monthly_premium_auto,number_open_complaints,total_claim_amount
0,185590,0,1000,0,3
1,697954,0,94,0,1131
2,1288743,48767,108,0,566
3,764586,0,106,0,530
4,536308,36357,68,0,17
...,...,...,...,...,...
8870,23406,71941,73,0,198
8871,3097,21604,79,0,379
8872,8164,0,85,3,791
8873,7524,21941,96,0,691


# Getting categorical columns

In [28]:
ca_df_standarized.select_dtypes("object")

,state,region,gender,education,policy_type,vehicle_class
0,washington,east,u,master,personal auto,four-door car
1,arizona,central,f,bachelor,personal auto,four-door car
2,nevada,central,f,bachelor,personal auto,two-door car
3,california,west,m,bachelor,corporate auto,suv
4,washington,east,m,high school or below,personal auto,four-door car
...,...,...,...,...,...,...
8870,california,west,m,bachelor,personal auto,four-door car
8871,california,west,f,college,corporate auto,four-door car
8872,california,west,m,bachelor,corporate auto,four-door car
8873,california,west,m,college,personal auto,four-door car
